# Assignment 3
Start with the sample machine learning program/code for hand-written digit recognition using MNIST dataset.
Change the network architecture of the machine learning model to increase the training accuracy of the model to not less than 99.30%.

## Prerequisite
Loading the MNIST dataset in Keras. The 'keras.datasets' module contains datasets that are used for machine learning tasks. The MNIST dataset is a large database of handwritten digits that is used for training image processing systems. The load_data() function loads the dataset into memory and returns a tuple for training data and one for testing data. 

Next the data is preprocessed, the reshape converts each 28x28 image into a single 784-element vector to prepare it for the fully connected layers. Then it changes the data type from integer to float to facilitate better computation in the neural network. And the dividing by 255 normalizes the pixel values to the range [0, 1] by dividing by 255 (the maximum value of a pixel).

One-hot encoding transforms categorical labels into a binary matrix where each class is represented by a vector with a single high (1) value and the rest low (0). The to_categorical function is used for this purpose where the train_labels are thee original labels in integer format and '10' is the number of classes. 

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow import keras

# Load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess
train_images = train_images.reshape((60000, 28, 28, 1)).astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype("float32") / 255

train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

## Model one 

### Model architecture
- Convolutional Layers: Three Conv2D layers with increasing filters (32, 64, 128) to capture spatial features.
- Batch Normalization: Applied after each Conv2D layer to stabilize and accelerate training.
- Pooling Layers: MaxPooling2D layers to reduce spatial dimensions and computational complexity.
- Global Average Pooling: Replaces Flatten to reduce the number of parameters and prevent overfitting.
- Dense Layers: A Dense layer with 128 units followed by a Dropout layer to prevent overfitting. Final Dense layer with 10 units for classification.
- Regularization: Dropout layer with a 0.4 rate to prevent overfitting. Early stopping and learning rate reduction to optimize training

In [7]:
import tensorflow as tf

def create_model_one():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),

        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.4),

        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model_one = create_model_one()
model_one.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,370 (435.04 KB)

 Trainable params: 110,922 (433.29 KB)

 Non-trainable params: 448 (1.75 KB)

### Training model one

In [8]:
import time

def train_model_one(model):

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    start_time = time.time()
    history = model.fit(train_images, train_labels, 
                        epochs=40, 
                        batch_size=256, 
                        validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr])

    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training stopped at epoch {early_stopping.stopped_epoch + 1}")
    print(f"Training time: {int(minutes)} minutes and {int(seconds)} seconds")
    
    return history


### Testing model one

In [9]:
model_one = create_model_one()   
history_one = train_model_one(model_one) 
test_loss, test_acc = model_one.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc}")

Epoch 1/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 42s 203ms/step - accuracy: 0.8481 - loss: 0.5611 - val_accuracy: 0.1060 - val_loss: 5.2147 - learning_rate: 0.0010
Epoch 2/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 66s 337ms/step - accuracy: 0.9832 - loss: 0.0585 - val_accuracy: 0.3114 - val_loss: 2.3516 - learning_rate: 0.0010
Epoch 3/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 62s 231ms/step - accuracy: 0.9885 - loss: 0.0383 - val_accuracy: 0.9436 - val_loss: 0.1793 - learning_rate: 0.0010
Epoch 4/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 49s 262ms/step - accuracy: 0.9920 - loss: 0.0265 - val_accuracy: 0.9833 - val_loss: 0.0561 - learning_rate: 0.0010
Epoch 5/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 50s 263ms/step - accuracy: 0.9936 - loss: 0.0200 - val_accuracy: 0.9896 - val_loss: 0.0401 - learning_rate: 0.0010
Epoch 6/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 46s 247ms/step - accuracy: 0.9954 - loss: 0.0154 - val_accuracy: 0.9923 - val_loss: 0.0289 - learning_rate: 0.0010
Epoch 7/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 42s 225ms/step - accuracy: 0.9

### Results
| Trainable parameters | Accuracy | Time taken by training    | Optimizer | Loss   | Epochs             | Batch Size |
|----------------------|----------|---------------------------|-----------|--------|--------------------|------------|
| 110,922              | 99.31%   | 23 minutes and 12 seconds | Adam      | 0.0253 | 28 (stopped early) | 256        |

## Model two

In [25]:
import tensorflow as tf

def create_model_two():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),

        keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model_two = create_model_two()
model_two.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_45 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_15     │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,882 (437.04 KB)

 Trainable params: 111,178 (434.29 KB)

 Non-trainable params: 704 (2.75 KB)

### Training model two

In [26]:
import time

def train_model_two(model):

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

    start_time = time.time()
    history = model.fit(train_images, train_labels, 
                        epochs=40, 
                        batch_size=256, 
                        validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr])

    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training stopped at epoch {early_stopping.stopped_epoch + 1}")
    print(f"Training time: {int(minutes)} minutes and {int(seconds)} seconds")
    
    return history

### Testing model two

In [27]:
model_two = create_model_two()   
history_two = train_model_two(model_two) 
test_loss, test_acc = model_two.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc}")

Epoch 1/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 37s 176ms/step - accuracy: 0.8587 - loss: 0.5859 - val_accuracy: 0.1060 - val_loss: 5.0469 - learning_rate: 0.0010
Epoch 2/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 32s 168ms/step - accuracy: 0.9816 - loss: 0.1629 - val_accuracy: 0.3362 - val_loss: 1.8166 - learning_rate: 0.0010
Epoch 3/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 31s 165ms/step - accuracy: 0.9885 - loss: 0.1134 - val_accuracy: 0.9166 - val_loss: 0.3296 - learning_rate: 0.0010
Epoch 4/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 31s 165ms/step - accuracy: 0.9910 - loss: 0.0842 - val_accuracy: 0.9843 - val_loss: 0.0962 - learning_rate: 0.0010
Epoch 5/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 174ms/step - accuracy: 0.9929 - loss: 0.0622 - val_accuracy: 0.9868 - val_loss: 0.0733 - learning_rate: 0.0010
Epoch 6/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 30s 161ms/step - accuracy: 0.9949 - loss: 0.0473 - val_accuracy: 0.9886 - val_loss: 0.0628 - learning_rate: 0.0010
Epoch 7/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 28s 150ms/step - accuracy: 0.9

### Results
| Trainable parameters | Accuracy | Time taken by training    | Optimizer | Loss   | Epochs             | Batch Size |
|----------------------|----------|---------------------------|-----------|--------|--------------------|------------|
| 111,178             | 99.39%   | 13 minutes and 19 seconds | Adam      | 0.0259 | 25 (stopped early) | 256        |

## Model Three

In [28]:
import tensorflow as tf

def create_model_three():
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.GlobalAveragePooling2D(),

        keras.layers.Flatten(),
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),

        keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.4),

        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model_three = create_model_three()
model_three.summary()

C:\Users\laris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_51 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_55          │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_56          │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_17     │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_57          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_58          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,314 (634.04 KB)

 Trainable params: 161,098 (629.29 KB)

 Non-trainable params: 1,216 (4.75 KB)

### Training model three

In [29]:
import time

def train_model_three(model):

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    start_time = time.time()
    history = model.fit(train_images, train_labels, 
                        epochs=40, 
                        batch_size=256, 
                        validation_split=0.2, 
                        callbacks=[early_stopping, reduce_lr])

    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training stopped at epoch {early_stopping.stopped_epoch + 1}")
    print(f"Training time: {int(minutes)} minutes and {int(seconds)} seconds")
    
    return history

### Testing model three

In [30]:
model_three = create_model_three()   
history_three = train_model_three(model_three) 
test_loss, test_acc = model_three.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc}")

C:\Users\laris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 48s 213ms/step - accuracy: 0.7978 - loss: 0.9872 - val_accuracy: 0.1060 - val_loss: 5.2523 - learning_rate: 0.0010
Epoch 2/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 44s 232ms/step - accuracy: 0.9748 - loss: 0.3582 - val_accuracy: 0.1576 - val_loss: 4.4078 - learning_rate: 0.0010
Epoch 3/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 44s 232ms/step - accuracy: 0.9845 - loss: 0.2566 - val_accuracy: 0.8962 - val_loss: 0.4817 - learning_rate: 0.0010
Epoch 4/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 47s 248ms/step - accuracy: 0.9884 - loss: 0.1855 - val_accuracy: 0.9824 - val_loss: 0.1704 - learning_rate: 0.0010
Epoch 5/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 42s 222ms/step - accuracy: 0.9916 - loss: 0.1306 - val_accuracy: 0.9868 - val_loss: 0.1256 - learning_rate: 0.0010
Epoch 6/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 44s 235ms/step - accuracy: 0.9912 - loss: 0.1025 - val_accuracy: 0.9708 - val_loss: 0.1637 - learning_rate: 0.0010
Epoch 7/40
188/188 ━━━━━━━━━━━━━━━━━━━━ 65s 347ms/step - accuracy: 0.9

### Results
| Trainable parameters | Accuracy | Time taken by training    | Optimizer | Loss   | Epochs             | Batch Size |
|----------------------|----------|---------------------------|-----------|--------|--------------------|------------|
| 161,098             | 99.36%   | 22 minutes and 58 seconds | Adam      | 0.0329 | 34 (stopped early) | 256        |